# API ROCrate Tests

In [406]:
import requests
import json
from requests_toolbelt.multipart.encoder import MultipartEncoder

root_url = "http://localhost:8080/"

## Create users

In [407]:
user1_data = {
    "@id": "ark:99999/test-user1",
    "name": "Test User1",
    "type": "Person",
    "email": "testuser1@example.org",
    "password": "test1",
    "organizations": [],
    "projects": [],
    "datasets": [],
    "rocrates": [],
    "software": [],
    "computations": [],
    "evidencegraphs": []
}
user1_create = requests.post(root_url + "user", json=user1_data)
user1_create.json()


{'error': 'document already exists'}

## Create Organization

In [408]:
org_data = {
    "@id": "ark:99999/test-org",
    "name": "test organization",
    "@type": "Organization",
    "owner": {
        "@id": user1_data['@id'],
        "name": user1_data['name'],
        "email": user1_data['email'],
        "@type": "Person"
    }
}

In [409]:
import base64
encoded_credentials = base64.b64encode(
    bytes(f"{user1_data.get('email')}:{user1_data.get('password')}", "utf-8"))

owner_headers = {"Authorization" : f"Basic {str(encoded_credentials, 'utf-8')}" }

organization_create = requests.post(root_url + "organization", data=json.dumps(org_data), headers=owner_headers)
organization_create.json()

{'error': 'organization already exists'}

## Create Project

In [410]:
project_data = {
    "@id": "ark:99999/test-org/test-proj",
    "name": "Test Project",
    "@type": "Project",
    "owner": {
        "@id": user1_data['@id'],
        "name": user1_data['name'],
        "email": user1_data['email'],
        "@type": "Person"
    },
    "memberOf": {
        "@id": org_data['@id'],
        "name": org_data['name'],
        "@type": org_data['@type']
    }
}
project_create = requests.post(root_url + "project", json=project_data, headers=owner_headers)
project_create.json()

{'error': 'project already exists'}

## Create ROCrate metadata

In [411]:
rocrate_data = {
    "@id": "ark:99999/UVA/B2AI/test_rocrate",
    "@type": "https://w3id.org/EVI#ROCrate",
    "name": "test-rocrate.zip",
    "isPartOf": [
        { 
            "@id": org_data['@id'], 
            "name": org_data['name'],
            "@type": org_data['@type']
        },
        {
            "@id": project_data['@id'],
            "name": project_data['name'],
            "@type": project_data['@type']
        }
    ],
    "@graph" : [
        {            
            "@id": "ark:/UVA/B2AI/test_rocrate/ro-crate-metadata.json",
            "metadataType": "https://w3id.org/EVI#Dataset",
            "conformsTo": {
              "@id": "https://w3id.org/ro/crate/1.1"
            },
            "about": {
              "@id": "ark:/UVA/B2AI/test_rocrate"
            },
            "isPartOf": {
              "@id": "ark:/UVA/B2AI/test_rocrate"
            },
            "contentUrl": "file://tests/guid_test_rocrate/ro-crate-metadata.json"            
        },
        {
          "@id": "ark:/UVA/B2AI/test_rocrate/AP-MS_embeddings-Dataset",
          "@context": {
            "@vocab": "https://schema.org/",
            "evi": "https://w3id.org/EVI#"
          },
          "metadataType": "https://w3id.org/EVI#Dataset",
          "name": "AP-MS embeddings",
          "author": "Gygi lab (https://gygi.hms.harvard.edu/team.html)",
          "datePublished": "2021-04-23",
          "version": "1.0.0",
          "description": "Affinity purification mass spectrometer (APMS) embeddings for each protein in the study,  generated by node2vec predict.",
          "associatedPublication": "Qin, Y. et al. A multi-scale map of cell structure fusing protein images and interactions",
          "additionalDocumentation": "https://idekerlab.ucsd.edu/music/",
          "format": "CSV",
          "schema": "",
          "generatedBy": [],
          "derivedFrom": [],
          "usedBy": [],
          "contentUrl": "file://tests/guid_test_rocrate/APMS_embedding_MUSIC.csv"
        }, 
        {
          "@id": "ark:59853/UVA/B2AI/rocrate_test/music_software",
          "@context": {
            "@vocab": "https://schema.org/",
            "evi": "https://w3id.org/EVI#"
          },
          "@type": "https://w3id.org/EVI#Software",
          "name": "MuSIC",
          "author": "Qin, Y.",
          "datePublished": "2021-06-20",
          "version": "1.0",
          "description": "script written in python to calibrate pairwise distance.",
          "associatedPublication": "Qin, Y. et al. A multi-scale map of cell structure fusing protein images and interactions. Nature 600, 536\u2013542 2021",
          "additionalDocumentation": "",
          #"format": ".py", # enabling instantiates as Dataset instead of Software
          "usedByComputation": [],
          "contentUrl": "file://tests/guid_test_rocrate/calibrate_pairwise_distance.py"    
        },
        {
          "@id": "ark:/UVA/B2AI/test_rocrate/average_predicted_protein_proximities-Computation",
          "@context": {
            "@vocab": "https://schema.org/",
            "evi": "https://w3id.org/EVI#"
          },
          "@type": "https://w3id.org/EVI#Computation",
          "name": "average predicted protein proximities",
          "runBy": "Max Levinson",
          "dateCreated": "03-17-2023",
          "description": "test run of music pipeline using example data",
          "associatedPublication": "",
          "additionalDocumentation": "",
          "command": "",
          "usedSoftware": [
            "random_forest_output (https://github.com/idekerlab/MuSIC/blob/master/random_forest_output.py)"
          ],
          "calledBy": "",
          "usedDataset": [
            "[predicted protein proximities:\nFold 1 proximities:\n    IF_emd_1_APMS_emd_1.RF_maxDep_30_nEst_1000.fold_1.pkl]"
          ],
          "generated": [
            "[averages of predicted protein proximities (https://github.com/idekerlab/MuSIC/blob/master/Examples/MuSIC_predicted_proximity.txt)]"
          ]          
        }
    ]
}

## Transfer ROCrate

In [412]:
mp_encoder = MultipartEncoder(
    fields={
        'rocrate': json.dumps(rocrate_data),        
        'file': ('test-rocrate', open('/home/sadnan/PycharmProjects/mds_python/tests/guid_test_rocrate.zip', 'rb'), 'application/zip '),
    }
)

# upload a rocrate to minio object store

rocrate_validate_transfer = requests.post(root_url + f"rocrate/publish",                              
    data=mp_encoder,  # The MultipartEncoder is posted as data, don't use files=...!
    # The MultipartEncoder provides the content-type header with the boundary:
    headers={'Content-Type': mp_encoder.content_type}
                )
                    
rocrate_validate_transfer.json()

{'created': {'@id': 'ark:99999/UVA/B2AI/test_rocrate',
  '@type': 'ROCrate',
  'name': 'test-rocrate.zip'}}